# Report generation actions

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * Action `report` generates plain text output and can be used to produce arbitrary log messages.
  * Action `pandoc` processes input document with pandoc
  * Action `Rmarkdown` process input document with Rmarkdown 
  

### Action `report`

Action `report` writes some content to an output stream. The input can either be a string or content of one or more files specified by option `input`. The output is determined by parameter `output`.

* If `output='filename'`, the content will be written to a file.
* If `output=obj` and `obj` has a `write` function (e.g. a file handle), the content will be passed to the `write` function
* If output is unspecified, the content will be written to standard output.

For example, the content of `report` actions is printed to standard output if no output is specified.

In [1]:
%run -v1

[10]
report: expand=True
    Runing {step_name}

[20]
report: expand=True
    Runing {step_name}


Runing 10

Runing 20



We can specify an output file with option `output`, but the output will be overwritten if multiple actions write to the same file

In [2]:
%run -v1
[10]
report: output='report.txt', expand=True
    Runing {step_name}

[20]
report: output='report.txt', expand=True
    Runing {step_name}

[30]
sh:
   cat report.txt

Runing 20



Action `report` can also take the content of one or more input files and write them to the output stream, after the script content (if specified). For example, the `report` action in the following example writes the content of `out.txt` to the default report stream (which is the standard output in this case).

In [3]:
%run -v1

[10]
output: 'out10.txt'
bash: expand=True
   # run some command and generate a out.txt
   echo "* result from step 10 " > {_output}


[20]
output: 'out20.txt'
bash: expand=True
   # run some command and generate a out.txt
   echo "* result from step 20 " > {_output}

[100]
report: input=['out10.txt', 'out20.txt']
Summary Report:

Summary Report:

* result from step 10

* result from step 20



### Action  `pandoc`

Action `pandoc` uses command [pandoc](http://pandoc.org/) to convert specified input to output. This input to this action can be specified from option `script` (usually specified in script format) and `input`.

First, if a script is specified, pandoc assumes it is in markdown format and convert it by default to 'HTML' format. For example,

In [4]:
pandoc:
    # this is header
    This is some test, with **emphasis**.        

<h1 id="this-is-header">this is header</h1>
<p>This is some test, with <strong>emphasis</strong>.</p>


You can specify an output with option `output`

In [5]:
%preview out.html
pandoc: output='out.html'
    Itemize

    * item 1
    * item 2

You can convert input file to another file type using a different file extension

In [6]:
%preview out.tex
pandoc: output='out.tex'
    Itemize

    * item 1
    * item 2

Or you can add more options to the command line by modifying `args`,

In [7]:
%preview out.html

pandoc: output='out.html', args='{input:q} --output {output:q} -s'    
    Itemize

    * item 1
    * item 2

[WARNING] This document format requires a nonempty <title> element.
  Please specify either 'title' or 'pagetitle' in the metadata,
  e.g. by using --metadata pagetitle="..." on the command line.
  Falling back to 'tmp9c9k9jhr'


The second usage of the `pandoc` action is to specify one or more input filenames. You have to use the function form of this action as follows

In [8]:
%preview out.html
[10]
report: output = 'out.md'
    Itemize

    * item 1
    * item 2

[20]
pandoc(input='out.md', output='out.html')
    

If multiple files are specified, the content of these input files will be concatenated. This is very useful for generating a single pandoc output with input from different steps. We will demonstrate this feature in the [Generating Reports](report_actions.html) tutorial.

If both `script` and `input` parameters are specified, the content of input files would be appended to `script`. So

In [9]:
%preview out.html
[10]
report: output = 'out10.md'
    Itemize

    * item 1
    * item 2

[20]
report: output= 'out20.md'
    enumerated

    1. item 1
    2. item 2

[30]
pandoc: input=['out10.md', 'out20.md'], output='out.html'
    Markdown supports both itemized and enumerated

## Action `Rmarkdown`

Action `Rmarkdown` is very similar to `pandoc`, the only difference is that it uses `RMarkdown` to process the input, which is assumed to be of Rmarkdown format.

In [10]:
%preview out.html
Rmarkdown: output='out.html'

Here's a numbered list:

 1. first item
 2. second item
 3. third item


/Users/bpeng1/anaconda3/envs/sos/bin/pandoc +RTS -K512m -RTS tmp8j3g4tz4.utf8.md --to html4 --from markdown+autolink_bare_uris+tex_math_single_backslash+smart --output /Users/bpeng1/sos/sos-docs/src/user_guide/out.html --email-obfuscation none --self-contained --standalone --section-divs --template /Users/bpeng1/anaconda3/envs/sos/lib/R/library/rmarkdown/rmd/h/default.html --no-highlight --variable highlightjs=1 --variable 'theme:bootstrap' --include-in-header /var/folders/ys/gnzk0qbx5wbdgm531v82xxljv5yqy8/T//RtmpTVT462/rmarkdown-str13d6f78c71598.html --mathjax --variable 'mathjax-url:https://mathjax.rstudio.com/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML' --lua-filter /Users/bpeng1/anaconda3/envs/sos/lib/R/library/rmarkdown/rmd/lua/pagebreak.lua --lua-filter /Users/bpeng1/anaconda3/envs/sos/lib/R/library/rmarkdown/rmd/lua/latex-div.lua 


[WARNING] This document format requires a nonempty <title> element.
  Please specify either 'title' or 'pagetitle' in the metadata,
  e.g. by using --metadata pagetitle="..." on the command line.
  Falling back to 'tmp8j3g4tz4.utf8'

Output created: out.html
